In [1]:
## Notebook to analyze the anode/cathode waveforms from the liquid purity monitor and calculate
## the outgassing rate

import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../WaveformAnalysis/')
import Dataset as ds

In [2]:
## path to data
data_path = '/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/nEXO/lifetime_analysis/data/20230804/'

## glob string for filenames to use
filename = 'LXe_C2700*.h5'

In [3]:
## intialize the dataset object and import data
dset = ds.Dataset(data_path, Selection=filename)
ch = dset.InitializeChannels()
for filename in dset.Files:
    dset.ImportDataFromHDF5(filename, ch)